Colab 音檔分析器 - 語音轉文字 + 情緒辨識

In [ ]:
!apt-get install -y ffmpeg
!pip install -q transformers torch librosa soundfile accelerate
!pip install -q openai-whisper pydub

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 35.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
import sys
import torch
import warnings
from IPython.display import display, Markdown, Audio, HTML
import json

warnings.filterwarnings("ignore")

# 環境檢查

In [ ]:
print("=" * 70)
print("🔍 系統環境檢查")
print("=" * 70)

required_packages = {
    "transformers": "transformers",
    "torch": "torch",
    "whisper": "openai-whisper",
    "soundfile": "soundfile",
    "librosa": "librosa",
    "pydub": "pydub"
}

missing = []
for module, package in required_packages.items():
    try:
        __import__(module)
        print(f"✅ {package}")
    except ImportError:
        missing.append(package)
        print(f"❌ {package} 未安裝")

if missing:
    print(f"\n⚠️  缺少套件: {', '.join(missing)}")
    sys.exit(1)

from google.colab import files
import whisper
from transformers import pipeline
import soundfile as sf
from pydub import AudioSegment
import librosa
import numpy as np

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n🖥️  運算設備: {DEVICE}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

print("=" * 70 + "\n")

🔍 系統環境檢查
✅ transformers
✅ torch
✅ openai-whisper
✅ soundfile
✅ librosa
✅ pydub

🖥️  運算設備: cuda
   GPU: Tesla T4



# 模型載入

In [ ]:
print("📦 正在載入 AI 模型...\n")

# 1. Whisper 語音辨識模型（使用更大的模型提升準確度）
whisper_model = None
WHISPER_SIZE = "medium"  # 從 base 升級到 medium，準確度大幅提升

print(f"1️⃣  Whisper 語音辨識模型 ({WHISPER_SIZE})")
print("   📝 使用 medium 模型以提升辨識準確度")
print("   ⏳ 載入中（可能需要 1-2 分鐘）...")

try:
    whisper_model = whisper.load_model(WHISPER_SIZE, device=DEVICE)
    print("   ✅ 載入成功\n")
except Exception as e:
    print(f"   ⚠️  {WHISPER_SIZE} 模型載入失敗，嘗試 small 模型...")
    try:
        WHISPER_SIZE = "small"
        whisper_model = whisper.load_model(WHISPER_SIZE, device=DEVICE)
        print("   ✅ small 模型載入成功\n")
    except Exception as e2:
        print(f"   ❌ 載入失敗: {e2}\n")

# 2. 多情緒分類模型（多維度分析）
emotion_classifier = None

print("2️⃣  多維度情緒分析模型")
print("   載入中...")

try:
    # 使用支援多情緒分類的模型
    emotion_classifier = pipeline(
        "text-classification",
        model="j-hartmann/emotion-english-distilroberta-base",
        device=0 if DEVICE == "cuda" else -1,
        return_all_scores=True  # 返回所有情緒分數
    )
    print("   ✅ 載入成功（支援 7 種情緒維度）\n")
except Exception as e:
    print(f"   ⚠️  嘗試備用模型...")
    try:
        emotion_classifier = pipeline(
            "sentiment-analysis",
            model="cardiffnlp/twitter-roberta-base-sentiment-latest",
            device=0 if DEVICE == "cuda" else -1,
        )
        print("   ✅ 備用模型載入成功\n")
    except Exception as e2:
        print(f"   ❌ 失敗: {e2}\n")

print("=" * 70)
print("✅ 模型載入完成")
print("=" * 70 + "\n")

📦 正在載入 AI 模型...

1️⃣  Whisper 語音辨識模型 (medium)
   📝 使用 medium 模型以提升辨識準確度
   ⏳ 載入中（可能需要 1-2 分鐘）...


100%|█████████████████████████████████████| 1.42G/1.42G [00:39<00:00, 39.0MiB/s]


   ✅ 載入成功

2️⃣  多維度情緒分析模型
   載入中...


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


   ✅ 載入成功（支援 7 種情緒維度）

✅ 模型載入完成



# 情緒映射資料庫

In [ ]:
EMOTION_DATABASE = {
    "anger": {
        "zh": "憤怒",
        "description": "充滿怒火、不滿和激動",
        "visual": "眉頭緊皺、咬牙切齒、臉色漲紅",
        "effects": "火焰、閃電、爆炸特效",
        "colors": ["深紅色", "橙紅色", "暗灰色"],
        "mood": "強烈、激烈、暴躁",
        "intensity_high": "暴怒、憤慨、火冒三丈",
        "intensity_mid": "生氣、不悅、煩躁",
        "intensity_low": "微慍、不快、皺眉"
    },
    "disgust": {
        "zh": "厭惡",
        "description": "反感、排斥和不適",
        "visual": "皺鼻子、嘴角下垂、後退姿態",
        "effects": "波浪線、X符號、迴避動作",
        "colors": ["深綠色", "紫灰色", "暗黃色"],
        "mood": "排斥、不舒服、嫌棄",
        "intensity_high": "極度反感、作嘔、避之不及",
        "intensity_mid": "厭煩、不喜歡、皺眉",
        "intensity_low": "輕微不適、有點嫌棄"
    },
    "fear": {
        "zh": "恐懼",
        "description": "害怕、緊張和不安",
        "visual": "眼睛睜大、身體顫抖、冷汗直流",
        "effects": "陰影、鋸齒背景、驚嚇線",
        "colors": ["深藍色", "紫色", "黑灰色"],
        "mood": "驚恐、膽怯、緊張",
        "intensity_high": "恐慌、驚駭、嚇壞了",
        "intensity_mid": "害怕、擔心、緊張",
        "intensity_low": "不安、微微擔憂"
    },
    "joy": {
        "zh": "喜悅",
        "description": "開心、快樂和興奮",
        "visual": "笑容燦爛、眼睛彎彎、手舞足蹈",
        "effects": "星星、愛心、閃光特效",
        "colors": ["明黃色", "粉紅色", "天藍色"],
        "mood": "歡樂、愉悅、雀躍",
        "intensity_high": "狂喜、欣喜若狂、樂開懷",
        "intensity_mid": "開心、快樂、滿足",
        "intensity_low": "微笑、愉快、心情好"
    },
    "neutral": {
        "zh": "平靜",
        "description": "冷靜、平和和中性",
        "visual": "表情自然、眼神平和、姿態放鬆",
        "effects": "簡約線條、幾何圖形",
        "colors": ["淡藍色", "米白色", "淺灰色"],
        "mood": "淡定、平穩、理性",
        "intensity_high": "非常冷靜、泰然自若",
        "intensity_mid": "平和、中性、淡然",
        "intensity_low": "略顯平淡"
    },
    "sadness": {
        "zh": "悲傷",
        "description": "難過、失落和憂鬱",
        "visual": "眼角下垂、嘴角下彎、低頭不語",
        "effects": "淚水、雨滴、陰雲",
        "colors": ["深藍色", "灰藍色", "淡紫色"],
        "mood": "哀傷、沮喪、憂鬱",
        "intensity_high": "極度悲痛、痛哭流涕、心碎",
        "intensity_mid": "難過、傷心、失落",
        "intensity_low": "有點難過、微微失落"
    },
    "surprise": {
        "zh": "驚訝",
        "description": "意外、震驚和吃驚",
        "visual": "眼睛睜大、嘴巴張開、身體後仰",
        "effects": "驚嘆號、閃電、爆炸效果",
        "colors": ["亮橙色", "黃色", "白色"],
        "mood": "驚奇、意外、震驚",
        "intensity_high": "極度震驚、目瞪口呆、不敢置信",
        "intensity_mid": "驚訝、吃驚、意外",
        "intensity_low": "有點意外、稍感驚奇"
    }
}

# 備用映射（用於不同模型）
EMOTION_FALLBACK = {
    "positive": "joy",
    "negative": "sadness",
    "POSITIVE": "joy",
    "NEGATIVE": "sadness",
    "NEUTRAL": "neutral"
}


# 核心分析函數

In [ ]:
def convert_audio_to_wav(audio_path):
    """轉換音檔格式"""
    file_ext = os.path.splitext(audio_path)[1].lower()
    if file_ext == '.wav':
        return audio_path

    print(f"   🔄 轉換 {file_ext} → WAV...")

    try:
        audio = AudioSegment.from_file(audio_path)
        wav_path = audio_path.rsplit('.', 1)[0] + '_converted.wav'
        audio.export(wav_path, format='wav')
        print(f"   ✅ 轉換完成")
        return wav_path
    except Exception as e:
        try:
            y, sr = librosa.load(audio_path, sr=None)
            wav_path = audio_path.rsplit('.', 1)[0] + '_converted.wav'
            sf.write(wav_path, y, sr)
            print(f"   ✅ 轉換完成（librosa）")
            return wav_path
        except Exception as e2:
            print(f"   ❌ 轉換失敗: {e2}")
            return None

def transcribe_audio_enhanced(audio_path):
    """增強版語音轉文字（提升準確度）"""

    if not whisper_model:
        return None

    print("\n" + "=" * 70)
    print("🎤 步驟 1: 高精度語音轉文字")
    print("=" * 70)

    try:
        # 使用優化參數提升準確度
        result = whisper_model.transcribe(
            audio_path,
            language="zh",  # 明確指定中文
            task="transcribe",
            fp16=(DEVICE == "cuda"),
            # 提升準確度的參數
            temperature=0.0,  # 降低隨機性，提高確定性
            beam_size=5,  # 使用 beam search
            best_of=5,  # 生成多個候選，選最佳
            patience=1.0,  # 耐心等待更好的結果
            condition_on_previous_text=True,  # 利用上下文
            initial_prompt="以下是普通話的句子。",  # 提示是中文
        )

        return result

    except Exception as e:
        print(f"❌ 轉錄失敗: {e}")
        return None

def analyze_emotions_multi(text):
    """多維度情緒分析"""

    if not emotion_classifier:
        return None

    print("\n" + "=" * 70)
    print("🧠 步驟 2: 多維度情緒分析")
    print("=" * 70)

    try:
        results = emotion_classifier(text[:512])

        # 處理返回格式
        if isinstance(results[0], list):
            # 多分數格式
            emotions = results[0]
            emotions_sorted = sorted(emotions, key=lambda x: x['score'], reverse=True)
            return emotions_sorted
        else:
            # 單一結果格式
            label = results[0]['label']
            score = results[0]['score']
            # 轉換為標準格式
            standard_label = EMOTION_FALLBACK.get(label, "neutral")
            return [{"label": standard_label, "score": score}]

    except Exception as e:
        print(f"❌ 情緒分析失敗: {e}")
        return None

def generate_emotion_description(emotions):
    """根據情緒分數生成詳細描述"""

    if not emotions:
        return "無法生成情緒描述"

    # 主要情緒
    primary = emotions[0]
    primary_label = primary['label']
    primary_score = primary['score']

    # 標準化標籤
    primary_label = EMOTION_FALLBACK.get(primary_label, primary_label)

    emotion_data = EMOTION_DATABASE.get(primary_label, EMOTION_DATABASE["neutral"])

    # 根據分數確定強度
    if primary_score >= 0.8:
        intensity = emotion_data["intensity_high"]
    elif primary_score >= 0.5:
        intensity = emotion_data["intensity_mid"]
    else:
        intensity = emotion_data["intensity_low"]

    # 次要情緒
    secondary_emotions = []
    if len(emotions) > 1:
        for emo in emotions[1:4]:  # 取前3個次要情緒
            if emo['score'] > 0.1:  # 分數大於10%才考慮
                label = EMOTION_FALLBACK.get(emo['label'], emo['label'])
                zh_name = EMOTION_DATABASE.get(label, {}).get("zh", label)
                secondary_emotions.append(f"{zh_name}({emo['score']:.0%})")

    # 組合描述
    description = f"""
**主要情緒**: {emotion_data['zh']} - {intensity}
**情緒強度**: {primary_score:.1%}
**情緒特徵**: {emotion_data['description']}
"""

    if secondary_emotions:
        description += f"**次要情緒**: {', '.join(secondary_emotions)}\n"

    description += f"""
**視覺表現**: {emotion_data['visual']}
**特效元素**: {emotion_data['effects']}
**建議配色**: {', '.join(emotion_data['colors'])}
**氛圍感受**: {emotion_data['mood']}
"""

    return description

def generate_sticker_prompts(text, emotions):
    """生成中英文生圖提示詞"""

    if not emotions:
        return {"chinese": "無法生成", "english": "Unable to generate"}

    primary = emotions[0]
    primary_label = EMOTION_FALLBACK.get(primary['label'], primary['label'])
    emotion_data = EMOTION_DATABASE.get(primary_label, EMOTION_DATABASE["neutral"])

    # 中文提示詞
    chinese_prompt = f"""## 🎨 AI 生圖提示詞（中文版）

**角色設計**:
- 可愛的 Q 版卡通角色，表現出【{emotion_data['zh']}】情緒
- 表情特徵：{emotion_data['visual']}
- 肢體語言：{emotion_data['mood']}的姿態

**視覺風格**:
- 風格：Line 貼圖風格、扁平化設計、粗黑邊框
- 配色方案：{', '.join(emotion_data['colors'])}為主色調
- 特效：{emotion_data['effects']}

**文字元素**:
- 標語文字：「{text}」
- 字體：粗體、清晰、易讀
- 位置：角色下方或周圍，與情緒相呼應

**畫面構成**:
- 背景：純色或漸層，{emotion_data['colors'][0]}系
- 尺寸：512x512 像素，正方形構圖
- 情緒強度：{primary['score']:.0%}，需要強烈表現
"""

    # 英文提示詞（保留原文）
    color_en = ", ".join(emotion_data['colors'])

    english_prompt = f"""## 🎨 AI Image Generation Prompt (English)

A cute chibi cartoon character expressing **{emotion_data['zh']} ({primary_label})** emotion.

**Character Design**:
- Facial expression: {emotion_data['visual']}
- Body language: showing {emotion_data['mood']} attitude
- Emotion intensity: {primary['score']:.0%}

**Visual Style**:
- Style: LINE sticker style, flat design, bold black outlines
- Color scheme: {color_en}
- Effects: {emotion_data['effects']}

**Text Element**:
- Speech bubble or caption with: "{text}"
- Font: Bold, clear, easy to read
- Position: Below or around the character

**Composition**:
- Background: Solid color or gradient, {emotion_data['colors'][0]} tone
- Size: 512x512 pixels, square format
- Overall mood: {emotion_data['mood']}

**Keywords**: cute, chibi, cartoon sticker, {primary_label}, expressive, messaging app, high quality, vector style
"""

    return {
        "chinese": chinese_prompt,
        "english": english_prompt
    }


主程式

In [ ]:
def main():
    """主程式"""

    if not whisper_model:
        display(Markdown("## ❌ Whisper 模型未載入"))
        return

    display(Markdown("""
# 🎙️ 音檔智能分析系統（高精度版）

## ✨ 功能特色：
- 🎯 **高精度語音辨識**：使用 medium 模型 + 優化參數
- 🧠 **多維度情緒分析**：7 種情緒維度完整分析
- 📝 **詳細情緒描述**：根據情緒數值生成視覺化描述
- 🎨 **雙語生圖提示詞**：中文版 + 英文版（含原文）

## 📤 請上傳音檔
支援：mp3, wav, m4a, flac, ogg, aac 等格式
"""))

    # 上傳
    try:
        uploaded = files.upload()
    except:
        return

    if not uploaded:
        return

    filename = list(uploaded.keys())[0]

    # 檢查檔案類型
    file_ext = os.path.splitext(filename)[1].lower()
    audio_extensions = ['.mp3', '.wav', '.m4a', '.flac', '.ogg', '.aac', '.wma', '.opus', '.webm']

    if file_ext not in audio_extensions:
        display(Markdown(f"""
## ❌ 檔案類型錯誤

您上傳的是 **{file_ext}** 檔案，這不是音檔格式！

### 📤 請上傳音檔：
- ✅ 支援格式：`.mp3`, `.wav`, `.m4a`, `.flac`, `.ogg`, `.aac` 等
- ❌ 不支援：`.png`, `.jpg`, `.pdf`, `.txt` 等非音檔格式

### 💡 如何取得音檔？
1. 使用手機錄音 APP 錄製語音
2. 從語音訊息匯出音檔
3. 使用線上文字轉語音工具

請重新上傳正確的音檔格式。
"""))
        os.remove(filename)
        return

    display(Markdown(f"### ✅ 已上傳音檔: `{filename}`\n"))

    # 嘗試播放音檔
    try:
        display(Audio(filename, autoplay=False))
    except Exception as e:
        display(Markdown(f"⚠️ 無法播放預覽（但會嘗試分析）: {e}\n"))

    # 轉換格式
    wav_path = convert_audio_to_wav(filename)
    if not wav_path:
        return

    # === 步驟 1: 語音轉文字 ===
    whisper_result = transcribe_audio_enhanced(wav_path)

    if not whisper_result:
        if wav_path != filename:
            os.remove(wav_path)
        os.remove(filename)
        return

    text = whisper_result["text"].strip()
    language = whisper_result.get("language", "未知")

    print(f"✅ 轉錄完成")
    print(f"   偵測語言: {language}")
    print(f"   文字長度: {len(text)} 字元")
    print(f"   使用模型: Whisper {WHISPER_SIZE}")

    display(Markdown(f"""
---
## 📝 語音轉文字結果

### 完整內容：
> **「{text}」**

**辨識資訊**:
- 偵測語言: {language}
- 字元數: {len(text)}
- 使用模型: Whisper {WHISPER_SIZE}（高精度）
"""))

    # === 步驟 2: 多維度情緒分析 ===
    emotions = analyze_emotions_multi(text)

    if not emotions:
        display(Markdown("### ⚠️ 情緒分析失敗"))
    else:
        print(f"✅ 情緒分析完成")
        print(f"   偵測到 {len(emotions)} 種情緒維度")

        # 顯示所有情緒分數
        display(Markdown("---\n## 🧠 多維度情緒分析結果\n"))

        # 情緒分數表格
        table_html = """
        <div style='padding: 20px; border-radius: 10px; background-color: transparent;'>
        <h3 style='color:#222;'>📊 情緒分數分布</h3>
        <table style='width:100%; border-collapse: collapse; margin-top: 10px; background-color: transparent;'>
        <tr style='background: linear-gradient(to bottom, #d9edf7, #cbe3f5); color:#000;'>
            <th style='padding: 12px; border: 1px solid #bbb; text-align: left;'>情緒</th>
            <th style='padding: 12px; border: 1px solid #bbb; text-align: left;'>分數</th>
            <th style='padding: 12px; border: 1px solid #bbb; text-align: left;'>視覺化</th>
        </tr>
        """

        # 🎨 顏色漸層配置：明顯區分各條
        color_palette = ["#28a745", "#007bff", "#ffc107", "#fd7e14", "#dc3545", "#6f42c1", "#17a2b8"]

        for i, emo in enumerate(emotions[:7]):
            label = EMOTION_FALLBACK.get(emo['label'], emo['label'])
            zh_name = EMOTION_DATABASE.get(label, {}).get("zh", label)
            score = emo['score']
            bar_width = int(score * 100)

            # 對比強烈的條狀圖樣式
            color = color_palette[i % len(color_palette)]
            bar_html = f"""
            <div style='
                width: 100%;
                height: 22px;
                background: linear-gradient(to right, {color}CC {bar_width}%, #f0f0f0 {bar_width}%);
                border-radius: 4px;
                position: relative;
                box-shadow: inset 0 0 3px rgba(0,0,0,0.2);
            '>
                <span style='
                    position: absolute;
                    left: 8px;
                    top: 1px;
                    font-size: 12px;
                    color: #000;
                '>{score:.1%}</span>
            </div>
            """

            table_html += f"""
            <tr style='background-color: transparent;'>
                <td style='padding: 10px; border: 1px solid #ddd;'><strong>{zh_name}</strong> ({label})</td>
                <td style='padding: 10px; border: 1px solid #ddd;'><strong>{score:.1%}</strong></td>
                <td style='padding: 10px; border: 1px solid #ddd;'>{bar_html}</td>
            </tr>
            """

        table_html += "</table></div>"
        display(HTML(table_html))



        # === 步驟 3: 情緒描述 ===
        display(Markdown("\n---\n## 💭 情緒特徵描述\n"))

        emotion_desc = generate_emotion_description(emotions)
        display(Markdown(emotion_desc))

        # === 步驟 4: 生圖提示詞 ===
        display(Markdown("\n---\n## 🎨 AI 生圖提示詞\n"))

        prompts = generate_sticker_prompts(text, emotions)

        display(Markdown(prompts["chinese"]))
        display(Markdown("\n---\n"))
        display(Markdown(prompts["english"]))

        display(Markdown("""
---
## 📥 使用建議

### 方法 1: 複製提示詞到 AI 繪圖工具
推薦平台：
- 🎨 [Leonardo.ai](https://leonardo.ai) - 免費，質量高
- 🎨 [Playground AI](https://playgroundai.com) - 免費
- 🎨 [Midjourney](https://midjourney.com) - 付費，最專業
- 🎨 [DALL-E 3](https://openai.com/dall-e-3) - ChatGPT Plus 內建

### 方法 2: 使用 Stable Diffusion
如需在 Colab 中直接生成，請告知我，我可以加入 SD 模組。

### 提示：
- ✅ 英文提示詞效果通常更好
- ✅ 可根據需求調整提示詞
- ✅ 建議生成多張後挑選最佳
"""))

    # 清理
    if wav_path != filename:
        os.remove(wav_path)
    os.remove(filename)
    print("\n✅ 處理完成，檔案已清理")

if __name__ == "__main__":
    main()


# 🎙️ 音檔智能分析系統（高精度版）

## ✨ 功能特色：
- 🎯 **高精度語音辨識**：使用 medium 模型 + 優化參數
- 🧠 **多維度情緒分析**：7 種情緒維度完整分析
- 📝 **詳細情緒描述**：根據情緒數值生成視覺化描述
- 🎨 **雙語生圖提示詞**：中文版 + 英文版（含原文）

## 📤 請上傳音檔
支援：mp3, wav, m4a, flac, ogg, aac 等格式


Saving b56094681ebb091df3959c7100da7930.mp3 to b56094681ebb091df3959c7100da7930.mp3


### ✅ 已上傳音檔: `b56094681ebb091df3959c7100da7930.mp3`


   🔄 轉換 .mp3 → WAV...
   ✅ 轉換完成

🎤 步驟 1: 高精度語音轉文字
✅ 轉錄完成
   偵測語言: zh
   文字長度: 4 字元
   使用模型: Whisper medium



---
## 📝 語音轉文字結果

### 完整內容：
> **「沒出息!」**

**辨識資訊**:
- 偵測語言: zh
- 字元數: 4
- 使用模型: Whisper medium（高精度）


...

## 🎨 AI 生圖提示詞（中文版）

**角色設計**:
- 可愛的 Q 版卡通角色，表現出【憤怒】情緒
- 表情特徵：眉頭緊皺、咬牙切齒、臉色漲紅
- 肢體語言：強烈、激烈、暴躁的姿態

**視覺風格**:
- 風格：Line 貼圖風格、扁平化設計、粗黑邊框
- 配色方案：深紅色, 橙紅色, 暗灰色為主色調
- 特效：火焰、閃電、爆炸特效

**文字元素**:
- 標語文字：「沒出息!」
- 字體：粗體、清晰、易讀
- 位置：角色下方或周圍，與情緒相呼應

**畫面構成**:
- 背景：純色或漸層，深紅色系
- 尺寸：512x512 像素，正方形構圖
- 情緒強度：68%，需要強烈表現



---


## 🎨 AI Image Generation Prompt (English)

A cute chibi cartoon character expressing **憤怒 (anger)** emotion.

**Character Design**:
- Facial expression: 眉頭緊皺、咬牙切齒、臉色漲紅
- Body language: showing 強烈、激烈、暴躁 attitude
- Emotion intensity: 68%

**Visual Style**:
- Style: LINE sticker style, flat design, bold black outlines
- Color scheme: 深紅色, 橙紅色, 暗灰色
- Effects: 火焰、閃電、爆炸特效

**Text Element**:
- Speech bubble or caption with: "沒出息!"
- Font: Bold, clear, easy to read
- Position: Below or around the character

**Composition**:
- Background: Solid color or gradient, 深紅色 tone
- Size: 512x512 pixels, square format
- Overall mood: 強烈、激烈、暴躁

**Keywords**: cute, chibi, cartoon sticker, anger, expressive, messaging app, high quality, vector style



---
## 📥 使用建議

### 方法 1: 複製提示詞到 AI 繪圖工具
推薦平台：
- 🎨 [Leonardo.ai](https://leonardo.ai) - 免費，質量高
- 🎨 [Playground AI](https://playgroundai.com) - 免費
- 🎨 [Midjourney](https://midjourney.com) - 付費，最專業
- 🎨 [DALL-E 3](https://openai.com/dall-e-3) - ChatGPT Plus 內建

### 方法 2: 使用 Stable Diffusion
如需在 Colab 中直接生成，請告知我，我可以加入 SD 模組。

### 提示：
- ✅ 英文提示詞效果通常更好
- ✅ 可根據需求調整提示詞
- ✅ 建議生成多張後挑選最佳



✅ 處理完成，檔案已清理
